# ДЗ 2 для финального проекта

Представим, что мы построили социальную сеть для студентов Karpov Courses, которая обладает следующим функционалом: можно отправлять друг другу письма, создавать сообщества, аналогичные группам в известных сетях, и в этих сообществах публиковать посты.

Из приятного – при регистрации студенты должны заполнять данные по своему профилю, которые хранятся в поднятой на наших мощностях postgres database.

Так же наша платформа обладает лентой, которую пользователи могут листать и просматривать случайные записи случайных сообществ. Если пост нравится, можно поддержать автора и поставить like.

Все действия пользователей сохраняются, каждая их активность, связанная с просмотром постов, тоже записывается к нам в базу.

Платформа Karpov Courses заинтересована в благосостоянии студентов, поэтому разработчики решили усовершенствовать текущую ленту. А что, если показывать пользователям не случайные посты, а рекомендовать их точечно каждому пользователю из всего имеющегося множества написанных постов? Как это сделать и учесть индивидуальные характеристики профиля пользователя, его прошлую активность и содержимое самих постов?

В текущем домашнем задании вам предстоит построить рекомендательную систему постов в социальной сети. В качестве базовых сырых данных вы будете использовать подготовленные заранее командой курса таблицы.

С точки зрения разработки вам будет необходимо реализовать сервис, который будет для каждого юзера в любой момент времени возвращать посты, которые пользователю покажут в его ленте соцсети.

### Оценка качества модели

Качество написанного вами алгоритма будет проверяться в чекере по скрытому для вас ряду user\_id и ряду timestаmp (эмулируем запросы пользователей в разное время) по метрике hitrate@5. Если вас испугала формула -- не переживайте. В дальнейших степах разберем эту метрику подробнее

**Hitrate@5**

$n⋅T1​∑t\=1T​∑i\=1n​min(1,∑j\=15​\[aj​(xi​,t)\=1\]),$

где:

- \* `n` - количество юзеров

- \* `T` - количество периодов проверки
- \* `aj​(xi​,t)` - j-ая рекомендация i-ому пользователю в момент времени t;

### Что мы проверяем в решении проекта

1. На практике мы хотим достаточно быстро формировать рекомендации. Поэтому будем требовать, чтобы алгоритм работал не более, чем ~0.5 секунд на 1 запрос, и занимал не более ~4 гб памяти (цифры приблизительные).
2. Набор юзеров фиксирован и никаких новых появляться не будет
3. Чекер будет проверять модель в рамках того же временного периода, что вы видите в БД
4. Модели не обучаются заново при использовании сервисов. Мы ожидаем, что ваш код будет импортировать уже обученную модель и применять ее.
5. В окружении стоят не все библиотеки, по мере выполнения вами ДЗ мы будем пополнять набор необходимых алгоритмов, текущая версия окружения приведена ниже. Для пополнения окружения достаточно написать в слак в службу поддержки.

```python
fastapi==0.75.1
pandas==1.4.2
sqlalchemy==1.4.35
requests==2.27.1
catboost==1.0.6
numpy==1.22.4
pydantic==1.9.1
scikit_learn==1.1.1
lightgbm==2.0.3
xgboost==1.6.1
psycopg2-binary==2.9.3
uvicorn==0.16.0
category-encoders==2.5.0
loguru==0.6.0
implicit==0.5.2
lightfm==1.16
```

### Несколько важных моментов

1. Просьба заранее сохранять код локально, он понадобится вам в следующей версии финального проекта в блоках про глубинное обучение и А/Б тестирование
2. Поддержка по техническим проблемам, не связанным с работой чекера, будет осуществляться в меньшем объеме, так как это задание часть финального проекта. Мы ожидаем большей самостоятельности.
3. Несмотря на то, что данный урок идет после раздела рекомендательных систем в данном задании стоит использовать все семейства алгоритмов, которые вы проходили в курсе

---

### Пример пайплайна, который мы реализуем, описанный простыми словами

1. Загрузка данных из БД в Jupyter Hub, обзор данных
2. Создание признаков и обучающей выборки. Например, могут быть использованы признаки о пользователе, тексты постов и прочие статистики
3. Тренировка модели на Jupyter Hub и оценка ее качества на валидационной выборке 
4. Сохранение модели 
5. Написание сервиса: загрузка модели -> получение признаков для модели по user\_id -> предсказание постов, которые лайкнут -> возвращение ответа. **Важно: для того, чтобы чекер отработал, необходимо загрузить и сервис, и модель одновременно.**
6. Загрузка в LMS в чекер

### Описание данных

##### Таблица user\_data

Cодержит информацию о всех пользователях соц.сети

<table class="table table-bordered"><tbody><tr><td>Field name</td><td>Overview</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">age</span></td><td>Возраст пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">city</span></td><td>Город пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">country</span></td><td>Страна пользователя (в профиле)</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">exp_group</span></td><td>Экспериментальная группа: некоторая зашифрованная категория</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">gender</span></td><td>Пол пользователя</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">id</span></td><td>Уникальный идентификатор пользователя</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">os</span></td><td>Операционная система устройства, с которого происходит пользование соц.сетью</td></tr><tr><td><b>source</b></td><td>Пришел ли пользователь в приложение с органического трафика или с рекламы</td></tr></tbody></table>

##### Таблица post\_text\_df

Содержит информацию о постах и уникальный ID каждой единицы с соответствующим ей текстом и топиком

<table class="table table-bordered"><tbody><tr><td><b>Field name</b></td><td><b>Overview</b></td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">id</span></td><td>Уникальный идентификатор поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">text</span></td><td>Текстовое содержание поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">topic</span></td><td>Основная тематика</td></tr></tbody></table>

##### Таблица feed\_data

Содержит историю о просмотренных постах для каждого юзера в изучаемый период.

**Внимание: Таблица ООООЧЕНЬ большая. Рекомендуется не загружать ее полностью, иначе все упадет**

<table class="table table-bordered"><tbody><tr><td><b>Field name</b></td><td><b>Overview</b></td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">timestamp</span></td><td>Время, когда был произведен просмотр</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">user_id</span></td><td>id пользователя, который совершил просмотр</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">post_id</span></td><td>id просмотренного поста</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">action</span></td><td>Тип действия: просмотр или лайк</td></tr><tr><td><span style="font-weight:600" data-token-index="0" class="notion-enable-hover" data-reactroot="">target</span></td><td>1 у просмотров, если почти сразу после просмотра был совершен лайк, иначе 0. У действий like пропущенное значение.</td></tr></tbody></table>

**Пример как можно забрать из них данные**

```SQL
SELECT * FROM public.user_data;
SELECT * FROM public.post_text_df;
SELECT * FROM public.feed_data limit 1000; --используем лимит тк данных очень много

### Построение модели

#### Выгрузка данных и что делать если их много

На практике данных часто оказывается так много, что не представляется возможным заджойнить все таблицы, загрузить их в Pandas в оперативную память компьютера, построить Pipeline с обработкой и выделением признаков и обучить модель на сотнях миллионов записей. Такая ситуация пугает, если мы ожидаем, что сервис при каждом поднятии будет полностью проходить весь цикл, описанный выше. Как оптимизировать процесс в условиях ограниченной памяти и времени?

Как скачать данные: для загрузки и чтения таблиц в своих схемах используйте старую БД из уроков по Python(URL для подключения: `postgresql://robot-startml-ro:pheiph0hahj1Vaif@`). Используйте `pd.read_sql` для чтения данных.

#### Оптимизация HitRate@5

Для начала напомним, как устроена метрика. Она принимает значение 1, если среди предложенных 5 рекомендаций хотя бы 1 получила в итоге like от пользователя. Даже если все 5 предложенных постов в итоге будут оценены пользователем, все равно hitrate будет равен 1. Метрика бинарная! В противном случае, если ни один из предложенных постов не был оценен пользователем, hitrate  принимает значение 0. Эту метрику мы хотим максимизировать.

Как обучать модель, которая в итоге будет максимизировать данную метрику? 

Это неочевидно по двум причинам:

- Во-первых, эта метрика не дифференцируемая (по аналогии с индикатором от отступа в задаче классификации)
- Во-вторых, непонятно, что в таблице feed\_data считать за «пять рекомендаций». В этом смысле фокус рекомендательных систем как раз состоит в том, что обычно такие задачи оказываются достаточно творческими, и приходится придумывать прокси-метрики. То есть оптимизировать нечто иное, а уже на основании результатов производить выбор лучших кандидатов на основании целевой метрики.

Не лишним будет порассуждать в терминах вероятностей. Когда наша вера в то, что пользователь лайкнет что-то из наших рекомендаций, будет максимальна? Когда мы порекомендуем ровно те посты, которые пользователь лайкнет с большей вероятностью!

#### Валидация

В рамках задачи вам необходимо будет предсказывать посты для тех же юзеров, что и есть в текущей выборке, но в будущем. Следовательно, валидацию стоит организовать таким же образом. 

**Написание кода загрузки модель**

После того, как вы обучили модель, необходимо ее сохранить. Несколько примеров как это можно сделать:

**Scikit-learn:**
```python
import pickle

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import Lasso

X_train = [[1, 2],
           [3, 10],
           [6, 7]]

y_train = [5, 6, 7]

pipe = Pipeline(
    [
        ('transformer',
         StandardScaler()),
                 
        ('estimator',
         Lasso())
    ]
)

pipe.fit(X_train, y_train)

filename = 'sklearn_model.pkl'
pickle.dump(pipe, open(filename, 'wb'))

loaded_model = pickle.load(open(filename, 'rb'))

loaded_model.predict(X_train)
```

**LigthGBM:**

Пример взят из документации: [тык](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html#lightgbm.Booster.save_model), [тык](https://lightgbm.readthedocs.io/en/latest/pythonapi/lightgbm.Booster.html)

```python
import lightgbm as lgb

lg_model = LGBMClassifier()

X_train = [[1, 2],
	   [3, 5],
           [7, 6]]

y_train = [1,
           0,
           0]

lg_model.fit(X_train, y_train)
lg_bmodel.booster_.save_model('mode.txt')

bst = lgb.Booster(model_file='mode.txt')

bst.predict(X_train)
```

#### Что нужно сделать в данном степе?

В данном степе вам необходимо:

1. Скачать часть данных из базы данных, поисследовать их
2. Обучить модель
3. Сохранить ее и загрузить. Костяк кода для загрузки модели, который стоит использовать есть ниже:

**Важный момент: Просьба не менять функцию get\_model\_path и использовать ее для корректной работы и загрузки внутри ЛМС**

```python
import os

def get_model_path(path: str) -> str:
    if os.environ.get("IS_LMS") == "1":  # проверяем где выполняется код в лмс, или локально. Немного магии
        MODEL_PATH = '/workdir/user_input/model'
    else:
        MODEL_PATH = path
    return MODEL_PATH

def load_models():
    model_path = get_model_path("/my/super/path")
    # LOAD MODEL HERE PLS :)
    pass
```

В чекер необходимо загрузить модель и код загрузки, состоящий из функций `load_models` и `get_model_path`. Внутри чекера мы проверим, что модель загружается и содержит внутри себя методы `predict` и `predict_proba`

**Важно:** чекер в лмс будет всегда хранить загруженную модель по следующему пути `/workdir/user_input/model`, НЕ ИЗМЕНЯЙТЕ ЕГО В ФУНКЦИИ.